In [1]:
import pandas as pd
import calc_pokecount
import rank
import time

In [2]:
df_field = pd.read_json('..\\data\\field.json')
df_pokemon = pd.read_json('..\\data\\pokemon.json')

In [3]:
df_pokemon

,1,2,3,4,5,6,7,8,9,10,...,460,462,468,470,471,475,700,702,759,760
name,Bulbasaur,Ivysaur,Venusaur,Charmander,Charmeleon,Charizard,Squirtle,Wartortle,Blastoise,Caterpie,...,Abomasnow,Magnezone,Togekiss,Leafeon,Glaceon,Gallade,Sylveon,Dedenne,Stufful,Bewear
type,dozing,dozing,dozing,snoozing,snoozing,snoozing,slumbering,slumbering,slumbering,dozing,...,slumbering,slumbering,snoozing,dozing,slumbering,slumbering,snoozing,snoozing,slumbering,slumbering
ancestor,1,1,1,4,4,4,7,7,7,10,...,459,81,175,133,133,280,133,None,759,759
field,"{'0': [{'id': 1, 'rank': 1}, {'id': 2, 'rank':...","{'0': [{'id': 1, 'rank': 8}, {'id': 2, 'rank':...","{'0': [{'id': 1, 'rank': 19}, {'id': 2, 'rank'...","{'0': [{'id': 1, 'rank': 1}, {'id': 2, 'rank':...","{'0': [{'id': 1, 'rank': 8}, {'id': 2, 'rank':...","{'0': [{'id': 1, 'rank': 19}, {'id': 2, 'rank'...","{'0': [{'id': 1, 'rank': 1}, {'id': 2, 'rank':...","{'0': [{'id': 1, 'rank': 8}, {'id': 2, 'rank':...","{'0': [{'id': 1, 'rank': 19}, {'id': 2, 'rank'...","{'0': [{'id': 1, 'rank': 0}, {'id': 2, 'rank':...",...,"{'3': [{'id': 1, 'rank': 11}, {'id': 2, 'rank'...","{'0': [{'id': 1, 'rank': 21}, {'id': 2, 'rank'...","{'0': [{'id': 1, 'rank': 20}, {'id': 2, 'rank'...","{'0': [{'id': 1, 'rank': 19}], '4': [{'id': 1,...","{'3': [{'id': 1, 'rank': 14}, {'id': 2, 'rank'...","{'4': [{'id': 1, 'rank': 17}, {'id': 2, 'rank'...","{'0': [{'id': 1, 'rank': 17}, {'id': 2, 'rank'...","{'0': [{'id': 1, 'rank': 11}, {'id': 2, 'rank'...","{'4': [{'id': 1, 'rank': 0}, {'id': 2, 'rank':...","{'4': [{'id': 1, 'rank': 9}, {'id': 2, 'rank':..."
style,"{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...",...,"{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res...","{'1': {'rarity': 1, 'isAtopBerry': False, 'res..."


In [3]:
df_field.head()

,index,name,ranks,powers
0,0,ワカクサ,"[0, 3118, 7171, 11693, 17149, 23385, 31492, 41...","[0, 965232, 2073695, 4626380, 8331417, 19563553]"
1,1,シアン,"[0, 4822, 11090, 18082, 26520, 36164, 48700, 6...","[0, 1574330, 3511234, 7162381, 13488264, 30491..."
2,2,トープ,"[0, 6885, 15835, 25817, 37865, 51635, 69534, 9...","[0, 1829448, 4626380, 9935824, 19563553, 43706..."
3,3,ウノハナ,"[0, 10486, 24118, 39323, 57673, 78645, 105909,...","[0, 2821389, 7162383, 15469188, 28772756, 6569..."
4,4,ラピスラズリ,"[0, 12938, 29756, 48515, 71156, 97031, 130668,...","[0, 3154032, 7733187, 16663802, 30491808, 6867..."


In [13]:
energy_power = 2000000
nemuke_power = energy_power * 100
spo_factor = 38000
field_name = "ラピスラズリ"
df_field = df_field.query("name == @field_name")


poke_count = calc_pokecount.pokecount(df_field,nemuke_power)

select_negaos = pd.DataFrame(index=range(0),columns=['name','rarity','isAtopBerry','researchExp','dreamShards','candy','spo','gid'])
field_index = df_field.index[0]
current_rank, rank_type, rank_num = rank.rank(df_field, energy_power)
sleep_type = 'dozing'       # dozing:うとうと snoozing:すやすや slumbering:ぐっすり

In [14]:
start = time.perf_counter() #計測開始

name_list = []
negao_list = []

for id, pokemon in df_pokemon.items():
    if pokemon.type != sleep_type or str(field_index) not in pokemon.field:
        continue
    
    for negao in pokemon.field[str(field_index)]:
        if negao['rank'] <= current_rank:
            negao_star = negao['id']
            name_list.append(pokemon['name'])
            negao_list.append([pokemon.style[str(negao_star)][key] for key in ['rarity', 'isAtopBerry', 'researchExp', 'dreamShards', 'candy', 'spo', 'gid']])

select_negaos = pd.DataFrame({'name': name_list, 'rarity': [x[0] for x in negao_list], 'isAtopBerry': [x[1] for x in negao_list], 'researchExp': [x[2] for x in negao_list], 'dreamShards': [x[3] for x in negao_list], 'candy': [x[4] for x in negao_list], 'spo': [x[5] for x in negao_list], 'gid': [x[6] for x in negao_list]})
# print(select_negaos)


#使えるSPO
total_spo = int(nemuke_power / spo_factor)
# print("抽選するSPO：",total_spo)

# 絶食用の寝顔
min_negaos = select_negaos[select_negaos['spo'] == select_negaos['spo'].min()]
min_negaos = min_negaos[min_negaos['gid'] == min_negaos['gid'].min()]
# print(min_negaos)

# poke_count - 1匹分のリサーチ
result_negaos = pd.DataFrame(columns=['name','rarity','isAtopBerry','researchExp','dreamShards','candy','spo','gid'])
consumption_spo = 0     # 消費SPO
for i in range(poke_count -1):
    if result_negaos['isAtopBerry'].any():
        # 抽選済み寝顔におなかの上寝がある場合、Falseかつ残SPO以下の寝顔を抽出
        filtered_negaos = select_negaos[(select_negaos['isAtopBerry'] == False) & (select_negaos['spo'] <= total_spo)]
    else:
        # 抽選済み寝顔におなかの上寝がない場合、残SPO以下の寝顔を抽出
        filtered_negaos = select_negaos[select_negaos['spo'] <= total_spo]

    if len(filtered_negaos) > 0:
        # 寝顔をランダムで選び、SPOを消費する
        choice_negaos = filtered_negaos.sample(n=1)
        result_negaos = pd.concat([result_negaos,choice_negaos])
        total_spo = int(total_spo - choice_negaos['spo'].values[0])
    else:
        # SPOを消費しきったら、絶食厳選対象の寝顔が選ばれる
        result_negaos = pd.concat([result_negaos,min_negaos])
        total_spo = int(total_spo - min_negaos['spo'].values[0])
    result_negaos.reset_index(drop=True, inplace=True)
print(poke_count)
print(result_negaos)
print("最終枠SPO：",total_spo)

# 残りの1商品を購入する
if result_negaos['isAtopBerry'].any():
    # 抽選済み寝顔におなかの上寝がある場合、Falseかつ残SPO以下の寝顔を抽出
    filtered_negaos = select_negaos[(select_negaos['isAtopBerry'] == False) & (select_negaos['spo'] <= total_spo)]
else:
    # 抽選済み寝顔におなかの上寝がない場合、残SPO以下の寝顔を抽出
    filtered_negaos = select_negaos[select_negaos['spo'] <= total_spo]

if len(filtered_negaos) > 0:
    # 買える商品があるときは残金計算する
    max_negaos = filtered_negaos[filtered_negaos['spo'] == filtered_negaos['spo'].max()]
    max_negaos = max_negaos[max_negaos['gid'] == max_negaos['gid'].min()]
    result_negaos = pd.concat([result_negaos,max_negaos])
    total_spo = int(total_spo - max_negaos['spo'].values[0])
else:
    # 買える商品がないときは固定のものをかう
    result_negaos = pd.concat([result_negaos,min_negaos])
    total_spo = int(total_spo - min_negaos['spo'].values[0])
result_negaos.reset_index(drop=True, inplace=True)

print(result_negaos.tail(1))


end = time.perf_counter()
print('{:.4f}'.format((end-start)))

8
        name rarity isAtopBerry researchExp dreamShards candy   spo  gid
0   Meganium      4        True        4379        7882    13  2029  381
1    Ivysaur      3       False        1043        2205     9   583   21
2    Psyduck      1       False           9          16     3     2  121
3    Ivysaur      3       False        1043        2205     9   583   21
4   Primeape      1       False         116         245     5    83  181
5  Dragonair      3       False        1804        3813     9  1008  478
6     Mankey      2       False          76         197     5    55  104
最終枠SPO： 920
       name rarity isAtopBerry researchExp dreamShards candy  spo gid
7  Venusaur      2       False        1289        2321    10  757  65
0.0229


In [6]:
import pandas as pd

#使えるSPO
total_spo = int(nemuke_power / spo_factor)
print("抽選するSPO：",total_spo)

# 絶食用の寝顔
min_negaos = select_negaos[select_negaos['spo'] == select_negaos['spo'].min()]
min_negaos = min_negaos[min_negaos['gid'] == min_negaos['gid'].min()]
# print(min_negaos)

# poke_count - 1匹分のリサーチ
result_negaos = pd.DataFrame(columns=['name','rarity','isAtopBerry','researchExp','dreamShards','candy','spo','gid'])
consumption_spo = 0     # 消費SPO
for i in range(poke_count -2):
    if result_negaos['isAtopBerry'].any():
        # 抽選済み寝顔におなかの上寝がある場合、Falseかつ残SPO以下の寝顔を抽出
        filtered_negaos = select_negaos[(select_negaos['isAtopBerry'] == False) & (select_negaos['spo'] <= total_spo)]
    else:
        # 抽選済み寝顔におなかの上寝がない場合、残SPO以下の寝顔を抽出
        filtered_negaos = select_negaos[select_negaos['spo'] <= total_spo]

    if len(filtered_negaos) > 0:
        # 寝顔をランダムで選び、SPOを消費する
        choice_negaos = filtered_negaos.sample(n=1)
        result_negaos = pd.concat([result_negaos,choice_negaos])
        total_spo = int(total_spo - choice_negaos['spo'].values[0])
    else:
        # SPOを消費しきったら、絶食厳選対象の寝顔が選ばれる
        result_negaos = pd.concat([result_negaos,min_negaos])
        total_spo = int(total_spo - min_negaos['spo'].values[0])
    result_negaos.reset_index(drop=True, inplace=True)

print(result_negaos)
print("最終枠SPO：",total_spo)

# 残りの1商品を購入する
if result_negaos['isAtopBerry'].any():
    # 抽選済み寝顔におなかの上寝がある場合、Falseかつ残SPO以下の寝顔を抽出
    filtered_negaos = select_negaos[(select_negaos['isAtopBerry'] == False) & (select_negaos['spo'] <= total_spo)]
else:
    # 抽選済み寝顔におなかの上寝がない場合、残SPO以下の寝顔を抽出
    filtered_negaos = select_negaos[select_negaos['spo'] <= total_spo]

if len(filtered_negaos) > 0:
    # 買える商品があるときは残金計算する
    max_negaos = filtered_negaos[filtered_negaos['spo'] == filtered_negaos['spo'].max()]
    max_negaos = max_negaos[max_negaos['gid'] == max_negaos['gid'].min()]
    result_negaos = pd.concat([result_negaos,max_negaos])
    total_spo = int(total_spo - max_negaos['spo'].values[0])
else:
    # 買える商品がないときは固定のものをかう
    result_negaos = pd.concat([result_negaos,min_negaos])
    total_spo = int(total_spo - min_negaos['spo'].values[0])
result_negaos.reset_index(drop=True, inplace=True)


print(result_negaos.tail(1))


抽選するSPO： 2564
         name rarity isAtopBerry researchExp dreamShards candy  spo  gid
0    Primeape      1       False         116         245     5   83  181
1  Bellsprout      1       False           9          16     3    2  106
2     Psyduck      4        True         517         930     8  240  350
3     Ivysaur      2       False         318         673     7  203   20
4     Ivysaur      3       False        1043        2205     9  583   21
5      Mankey      3       False         324         840     7  203  105
最終枠SPO： 1250
        name rarity isAtopBerry researchExp dreamShards candy   spo  gid
6  Dragonair      3       False        1804        3813     9  1008  478
